In [ ]:
from droid_speak import (
    generate_tone_dictionaries,
    Encoder,
    Decoder,
    MIN_INDEX_DIFFERENCE,
    NUM_FREQUENCIES,
    MIN_FREQ,
    MAX_FREQ,
    NUM_WORDS_NEEDED ,
    SAMPLE_RATE,
    FADE_DURATION,
    VOLUME,
    WHISTLE_THRESHOLD,
    CHIRP_THRESHOLD,
    SEED,
    TONE_LENGTH,
    WHISTLE_LENGTH
)

# generate dictionaries
common_words_dict, character_dict = generate_tone_dictionaries(SEED, NUM_WORDS_NEEDED, NUM_FREQUENCIES, MIN_FREQ, MAX_FREQ, MIN_INDEX_DIFFERENCE)

# Create an encoder instance
encoder = Encoder(
    common_words_dict, 
    character_dict, 
    tone_length=TONE_LENGTH, 
    sample_rate=44100, 
    fade_duration=0.01, 
    volume=0.7)

# Encode a sentence
audio_sequence = encoder.encode_sentence("Hello World!!!")

# Play the encoded audio
encoder.play_audio(audio_sequence)

# Create a decoder instance
decoder = Decoder(
    sample_rate=44100,
    tone_length=TONE_LENGTH,
    whistle_length=WHISTLE_LENGTH,
    WHISTLE_THRESHOLD=WHISTLE_THRESHOLD,
    CHIRP_THRESHOLD=CHIRP_THRESHOLD,
    common_words_dict=common_words_dict,
    character_dict=character_dict
)

# Decode the audio sequence
decoded_text = decoder.decode_audio_sequence(audio_sequence)

print("Decoded text:", decoded_text)

## Autogen Droid Conversation

In [ ]:
# CLI steps before this code:
# pip install autogen
# pip install pyautogen
# pip install 'litellm[proxy]’
# ollama run llama3.1
# new tab: litellm –-model ollama/llama3.1
# copy url as base_url in config list ( http://0.0.0.0:4000 )

import os
from autogen import ConversableAgent

local_llm_config = [
    {
        "model": "NotRequired",  
        "api_key": "NotRequired",  
        "base_url": "http://127.0.0.1:4000"  # Your LiteLLM URL
    }
]

In [ ]:
import time
import logging
logging.getLogger("autogen.oai.client").setLevel(logging.ERROR)
logging.disable(logging.CRITICAL)

# Create the Conversable Agents
droid_1 = ConversableAgent(
    name="LB-1",
    system_message="You are an astromech droid from Star Wars working with another droid called J2M2."
    "You are discussing how to fix the hyperdrive. Droidspeak is slow, so use only short 5-6 word long sentences only.",
    llm_config={"config_list": local_llm_config},
    human_input_mode="NEVER",
)

droid_2 = ConversableAgent(
    name="J2M2",
    system_message="You are an astromech droid from Star Wars working with another droid called J2M2."
    "You are discussing how to fix the hyperdrive. Droidspeak is slow, so use only short 5-6 word long sentences only.",
    llm_config={"config_list": local_llm_config},
    human_input_mode="NEVER",
)

def simulate_droid_conversation(agent1, agent2, max_turns=3):
    turn = 0
    current_agent = agent1
    recipient_agent = agent2
    message = "We need to fix the hyperdrive!"
    
    while turn < max_turns:
        # Encode the message with the encoder
        encoded_message = encoder.encode_sentence(message)
        
        # Play the encoded message
        encoder.play_audio(encoded_message)
        
        # Decode the encoded message back to text
        decoded_message = decoder.decode_audio_sequence(encoded_message)
        
        # The recipient responds based on the decoded message
        response = recipient_agent.initiate_chat(
            recipient=current_agent,
            message=decoded_message,
            max_turns=1,
        )
        
        # Extract the new message from chat_history
        if response.chat_history and isinstance(response.chat_history, list):
            latest_message = response.chat_history[-1]
            if "content" in latest_message:
                message = latest_message["content"]
            else:
                raise ValueError("Unexpected structure in chat history entry.")
        else:
            raise ValueError("Chat history is missing or malformed.")
        
        # Print the newly received message from the other droid
        print(f"{recipient_agent.name} says:\n{message}\n{'-' * 80}")
        time.sleep(3)  # 3-second pause
        
        # Swap roles for next turn
        current_agent, recipient_agent = recipient_agent, current_agent
        turn += 1

# Run the simulation
simulate_droid_conversation(droid_1, droid_2)